## Million Songs Dataset - Using SQLite Databases 

There are five databases available to us:

* [track_metadata.db](http://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/track_metadata.db) contains most metadata about each track
* [artist_term.db](http://www.ee.columbia.edu/~thierry/artist_term.db) links artists id's to the tags
* [artist_similarity.db](http://www.ee.columbia.edu/~thierry/artist_similarity.db) contains similarity among artists
* [mxm_dataset.db](http://labrosa.ee.columbia.edu/millionsong/sites/default/files/AdditionalFiles/mxm_dataset.db) has lyrics for many of the tracks
   
Information about the first three can be found at http://labrosa.ee.columbia.edu/millionsong/pages/getting-dataset#subset 

For information about the musiXmatch database go to http://labrosa.ee.columbia.edu/millionsong/musixmatch

In [36]:
import pandas as pd
import sqlite3
from nltk.corpus import stopwords

We first need to create a connection to each of our database files. Each of the files is stored in the same folder as this document. We'll use the connection every time we query the database.

In [37]:
conn_tracks = sqlite3.connect('../Data/track_metadata.db')
conn_terms = sqlite3.connect('../Data/artist_term.db')
conn_similarity = sqlite3.connect('../Data/artist_similarity.db')
conn_lyrics = sqlite3.connect('../Data/mxm_dataset.db')

After we've created connections, there are a couple of ways we can pull data from the databases:

1. We can use SQLite to create a cursor which then fetches records based on our query parameters. A cursor is a database object used to traverse records in a database. The .execute method creates a cursor and then calls the cursors execute method. It returns the results as a list.

2. We can use the pandas read.sql function. It returns the results as a pandas DataFrame object

First, we'll want to look at the database schema. We'll start off by querying the sqlite_master table from a single sqlite database file. The sqlite_master table defines the schema for the database. We'll use that to get information about the database and create our first query. More information about the sqlite_master table can be found at view-source:https://www.sqlite.org/faq.html#q7

In [38]:
res = conn_tracks.execute("SELECT * FROM sqlite_master WHERE type = 'table'")
schema1 = res.fetchall()
type(schema1)

list

In [39]:
schema1

[('table',
  'songs',
  'songs',
  2,
  'CREATE TABLE songs (track_id text PRIMARY KEY, title text, song_id text, release text, artist_id text, artist_mbid text, artist_name text, duration real, artist_familiarity real, artist_hotttnesss real, year int, track_7digitalid int, shs_perf int, shs_work int)')]

We can also wrap this in a pandas DataFrame object. 

In [40]:
pd.DataFrame(schema1)

,0,1,2,3,4
0,table,songs,songs,2,"CREATE TABLE songs (track_id text PRIMARY KEY,..."


In [41]:
schema2 = pd.read_sql('SELECT * FROM sqlite_master where type="table"', con=conn_tracks)
type(schema2)

pandas.core.frame.DataFrame

In [42]:
schema2

,type,name,tbl_name,rootpage,sql
0,table,songs,songs,2,"CREATE TABLE songs (track_id text PRIMARY KEY,..."


For the schema and for some exploration, it makes more sense to pull it as a list, but when we start pulling data from tables, we'll want to use the  pandas read_sql() function, which retains the column headers. Next, we'll explore each of the databases available to us.

## track_metadata

We've seen that there is a single table in track metadata, called songs. Let's find out more about the table. We use our tracks connection to pull data from the songs table. We use fetchall(), which fetches all of the records. We'll use fetchall to get the length of the list.

Lets fetch a row from the songs table and find out how long it is. 

In [43]:
res = conn_tracks.execute("SELECT * FROM songs")
len(res.fetchall()) 

1000000

Now lets look at the list itself.

In [44]:
res.fetchall()

[]

We come up with an empty list. Why? 

It's because there are no remaining rows to fetch using that cursor. When we used the cursor to determine the length of our list, we traversed the entire list. We need to re-initialize the cursor to look at the values in words. We'll save the list to a variable this time.

In [45]:
res = conn_tracks.execute("SELECT * FROM songs")
tracks = res.fetchall()

In [46]:
len(tracks), tracks[0]

(1000000,
 ('TRMMMYQ128F932D901',
  'Silent Night',
  'SOQMMHC12AB0180CB8',
  'Monster Ballads X-Mas',
  'ARYZTJS1187B98C555',
  '357ff05d-848a-44cf-b608-cb34b5701ae5',
  'Faster Pussy cat',
  252.05506,
  0.649822100201,
  0.394031892714,
  2003,
  7032331,
  -1,
  0))

We can also look at a single track using fetchone(), which fetches the next record.

In [47]:
res = conn_tracks.execute("SELECT * FROM songs")
res.fetchone(), res.fetchone()

(('TRMMMYQ128F932D901',
  'Silent Night',
  'SOQMMHC12AB0180CB8',
  'Monster Ballads X-Mas',
  'ARYZTJS1187B98C555',
  '357ff05d-848a-44cf-b608-cb34b5701ae5',
  'Faster Pussy cat',
  252.05506,
  0.649822100201,
  0.394031892714,
  2003,
  7032331,
  -1,
  0),
 ('TRMMMKD128F425225D',
  'Tanssi vaan',
  'SOVFVAK12A8C1350D9',
  'Karkuteillä',
  'ARMVN3U1187FB3A1EB',
  '8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9',
  'Karkkiautomaatti',
  156.55138,
  0.439603966677,
  0.356992107756,
  1995,
  1514808,
  -1,
  0))

Using fetchone()[i] can be used to extract the ith element from the record. 

In [48]:
res = conn_tracks.execute("SELECT * FROM songs")
res.fetchone()[1]

'Silent Night'

Finally, lets read all of the data into a database we can use.

In [49]:
songs = pd.read_sql("SELECT * FROM SONGS", con = conn_tracks)

In [50]:
songs.head(5)

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0
3,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003,2168257,-1,0
4,TRMMMWA128F426B589,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Rene Ablaze Presents Winter Sessions,AREQDTE1269FB37231,,Der Mystic,514.29832,0.000000,0.000000,0,2264873,-1,0


## artist_term

The first thing we'll do is look at the table structure.

MOER ABOUT WHATS HERE

In [51]:
res = conn_terms.execute("Select * FROM sqlite_master where type = 'table'")
res.fetchall()

[('table',
  'artists',
  'artists',
  2,
  'CREATE TABLE artists (artist_id text PRIMARY KEY)'),
 ('table',
  'terms',
  'terms',
  2459,
  'CREATE TABLE terms (term text PRIMARY KEY)'),
 ('table',
  'artist_term',
  'artist_term',
  2782,
  'CREATE TABLE artist_term (artist_id text, term text, FOREIGN KEY(artist_id) REFERENCES artists(artist_id), FOREIGN KEY(term) REFERENCES terms(term) )'),
 ('table',
  'mbtags',
  'mbtags',
  2783,
  'CREATE TABLE mbtags (mbtag text PRIMARY KEY)'),
 ('table',
  'artist_mbtag',
  'artist_mbtag',
  2876,
  'CREATE TABLE artist_mbtag (artist_id text, mbtag text, FOREIGN KEY(artist_id) REFERENCES artists(artist_id), FOREIGN KEY(mbtag) REFERENCES mbtags(mbtag) )')]

We can see that there are 5 tables in this database: artists, terms, artist_term, mbtags, and artist_mbtag

The artists table is simply a list of all of the artists represented in the database. There are 44,745 artists in the database. 

In [52]:
artists = pd.read_sql("SELECT * FROM artists", con=conn_terms)
len(artists), artists.head(5)

(44745,             artist_id
 0  AR002UA1187B9A637D
 1  AR003FB1187B994355
 2  AR006821187FB5192B
 3  AR009211187B989185
 4  AR009SZ1187B9A73F4)

The terms table contains 7643 different terms. 

From http://labrosa.ee.columbia.edu/millionsong/pages/what-difference-between-terms-and-mbtags: "Terms" are the tags provided by The Echo Nest. They can come from a number of places, but mostly blogs as far as we understand.

In [53]:
terms = pd.read_sql("SELECT * FROM terms", con=conn_terms)
len(terms), terms.tail(10)

(7643,            term
 7633       zulu
 7634        zum
 7635     zurich
 7636       zxzw
 7637  zxzw 2005
 7638  zxzw 2006
 7639  zxzw 2007
 7640  zxzw 2008
 7641  zxzw 2009
 7642     zydeco)

The artist_term table links artists and terms. There are 1,109,381 rows in this table. 

In [54]:
artist_term = pd.read_sql("SELECT * FROM artist_term", con=conn_terms)
len(artist_term), artist_term.head(10)

(1109381,             artist_id          term
 0  AR002UA1187B9A637D   garage rock
 1  AR002UA1187B9A637D  country rock
 2  AR002UA1187B9A637D     free jazz
 3  AR002UA1187B9A637D            oi
 4  AR002UA1187B9A637D    space rock
 5  AR002UA1187B9A637D     power pop
 6  AR002UA1187B9A637D    indie rock
 7  AR002UA1187B9A637D        reggae
 8  AR002UA1187B9A637D      pop rock
 9  AR002UA1187B9A637D          rock)

We use the sum function to find the number of terms associated with artist "AR002UA1187B9A637D".

In [55]:
sum(artist_term['artist_id'] == 'AR002UA1187B9A637D')

27

Next we look at the terms associated with artist "AR002UA1187B9A637D".

In [56]:
artist_term.loc[artist_term['artist_id'] == 'AR002UA1187B9A637D']

,artist_id,term
0,AR002UA1187B9A637D,garage rock
1,AR002UA1187B9A637D,country rock
2,AR002UA1187B9A637D,free jazz
3,AR002UA1187B9A637D,oi
4,AR002UA1187B9A637D,space rock
5,AR002UA1187B9A637D,power pop
6,AR002UA1187B9A637D,indie rock
7,AR002UA1187B9A637D,reggae
8,AR002UA1187B9A637D,pop rock
9,AR002UA1187B9A637D,rock


The mbtags table contains 2321 terms. 

"Mbtags" are musicbrainz tags, specifically applied by humans to a particular artist. This explains why there are fewer of them (see 'mbtags_count'), but they are usually very clean and informative. For instance, if you want to create a genre recognition task where classes are mutually exclusive, mbtags are likely to be more reliable then terms.

In [57]:
mbtags = pd.read_sql("SELECT * FROM mbtags", con=conn_terms)
len(mbtags), mbtags[200:219]

(2321,               mbtag
 200       aylesbury
 201      b00138f5lm
 202            bach
 203         bachata
 204     bad nauheim
 205   bahasa melayu
 206       bajofondo
 207          balady
 208        balearic
 209          balkan
 210          ballad
 211         ballads
 212       baltimore
 213            band
 214           banda
 215  banda el limon
 216      bandleader
 217    bandoneonist
 218         barbade)

Finally, the artist_mbtag table associates artists with mbtags.

In [58]:
artist_mbtag = pd.read_sql("SELECT * FROM artist_mbtag", con=conn_terms)
len(artist_mbtag), artist_mbtag.head(10)

(24777,             artist_id        mbtag
 0  AR002UA1187B9A637D           uk
 1  AR002UA1187B9A637D         rock
 2  AR002UA1187B9A637D  garage rock
 3  AR006821187FB5192B         bass
 4  AR00A6H1187FB5402A      detroit
 5  AR00A6H1187FB5402A         punk
 6  AR00A6H1187FB5402A     michigan
 7  AR00A6H1187FB5402A          usa
 8  AR00B1I1187FB433EB     european
 9  AR00B1I1187FB433EB      swedish)

We see that, using mbtags returns fewer results than using terms. 

In [59]:
artist_mbtag.loc[artist_mbtag['artist_id'] == 'AR002UA1187B9A637D']

,artist_id,mbtag
0,AR002UA1187B9A637D,uk
1,AR002UA1187B9A637D,rock
2,AR002UA1187B9A637D,garage rock


## artist_similarity

There is are two tables in the artist_similarity database: artists and similarity.

In [60]:
res = conn_similarity.execute("Select * FROM sqlite_master where type = 'table'")
res.fetchall()

[('table',
  'artists',
  'artists',
  2,
  'CREATE TABLE artists (artist_id text PRIMARY KEY)'),
 ('table',
  'similarity',
  'similarity',
  2459,
  'CREATE TABLE similarity (target text, similar text, FOREIGN KEY(target) REFERENCES artists(artist_id), FOREIGN KEY(similar) REFERENCES artists(artist_id) )')]

There are 44,745 artists in the artists table, the same number of rows in the artist_terms database.

In [61]:
artists_s = pd.read_sql("SELECT * FROM artists", con = conn_similarity)


In [62]:
len(artists_s)

44745

In [63]:
similarity = pd.read_sql("SELECT * FROM similarity", con = conn_similarity)

There are 2,201,916 rows in the similarity table.

In [64]:
len(similarity)

2201916

Using the same artist we used in exploring the artist_term database, we find the artist is similar to 46 other artists.

In [65]:
similarity.loc[similarity['target'] == 'AR002UA1187B9A637D']

,target,similar
0,AR002UA1187B9A637D,ARQDOR81187FB3B06C
1,AR002UA1187B9A637D,AROHMXJ1187B989023
2,AR002UA1187B9A637D,ARAGWVR1187B9B749B
3,AR002UA1187B9A637D,AREQVWS1241B9CC0A4
4,AR002UA1187B9A637D,ARHBE351187FB3B0CD
5,AR002UA1187B9A637D,AR2EMEJ1187FB3BB2B
6,AR002UA1187B9A637D,ARIQN6P11C8A416C39
7,AR002UA1187B9A637D,ARHBUBO11F50C4EBAF
8,AR002UA1187B9A637D,AROULNY11F50C4EB79
9,AR002UA1187B9A637D,ARD58Q21187FB36F09


## mxm_dataset

The mxm_dataset has two tables: words and lyrics. 

The words table has a single column: word

The lyrics table has five columns:
track_id: MSD song id
mxm_tid: musiXmatch track id
word: one of the words in the table words
count: the word count for that track
is_test: tells you if a track is in the test set(1) or not(0)

In [66]:
res = conn_lyrics.execute("Select * FROM sqlite_master where type = 'table'")
res.fetchall()

[('table', 'words', 'words', 2, 'CREATE TABLE words (word TEXT PRIMARY KEY)'),
 ('table',
  'lyrics',
  'lyrics',
  4,
  'CREATE TABLE lyrics (track_id, mxm_tid INT, word TEXT, count INT, is_test INT, FOREIGN KEY(word) REFERENCES words(word))')]

There are 5000 unique words in the words table. We look at the first 5.

In [67]:
words = pd.read_sql('SELECT * FROM words', con=conn_lyrics)

In [68]:
len(words), words.head(5)

(5000,   word
 0    i
 1  the
 2  you
 3   to
 4  and)

The lyrics table contains the most data of all the tables we've looked at. There are 19,045,332 rows. This is because each row contains two track id's, a word, a count of the number of times the word appears, and a field indicating whether its part of the train or test set. 

In [69]:
lyrics = pd.read_sql("SELECT * FROM lyrics", con=conn_lyrics)

In [70]:
len(lyrics)

19045332

In [71]:
lyrics.head(5)

,track_id,mxm_tid,word,count,is_test
0,TRAAAAV128F421A322,4623710,i,6,0
1,TRAAAAV128F421A322,4623710,the,4,0
2,TRAAAAV128F421A322,4623710,you,2,0
3,TRAAAAV128F421A322,4623710,to,2,0
4,TRAAAAV128F421A322,4623710,and,5,0


In [72]:
lyrics.loc[lyrics['track_id'] == 'TRAAAAV128F421A322']

,track_id,mxm_tid,word,count,is_test
0,TRAAAAV128F421A322,4623710,i,6,0
1,TRAAAAV128F421A322,4623710,the,4,0
2,TRAAAAV128F421A322,4623710,you,2,0
3,TRAAAAV128F421A322,4623710,to,2,0
4,TRAAAAV128F421A322,4623710,and,5,0
5,TRAAAAV128F421A322,4623710,a,3,0
6,TRAAAAV128F421A322,4623710,me,1,0
7,TRAAAAV128F421A322,4623710,it,1,0
8,TRAAAAV128F421A322,4623710,my,1,0
9,TRAAAAV128F421A322,4623710,is,2,0


Last, we close our database connections. 

In [73]:
conn_tracks.close()
conn_terms.close()
conn_similarity.close()
conn_lyrics.close()

## Next Steps

The next step would be to integrate this data with the data we already have.
